# Finding the Best Bars in Ocean City

## Introduction
My friends and I are looking to go to Ocean City Maryland when COVID-19 slows down. The objective here is to find the best bar to go to each night to meet people

## Data
We will utilize Foursquare location data, which includes the latitude and longitude of various locations. The information below can be retrieved from the API when called. Any valid address can be used to retrieve the data.

## Methodology
We will scrape location data using an API and determine using the reviews of nearby bars which one is the best bar

In [118]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

from folium import features

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [74]:
! pip install folium

You are using pip version 9.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [76]:
CLIENT_ID = 'TTKHJVXTE44UI1ZDXT4VQN52NNAIG1MRF1SDHTUIKSQB1BK0' # your Foursquare ID
CLIENT_SECRET = 'ZGXCTAJU03PHASRDXRCVMQPCSITCFFZUYORPEKMNKISMFEKW' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TTKHJVXTE44UI1ZDXT4VQN52NNAIG1MRF1SDHTUIKSQB1BK0
CLIENT_SECRET:ZGXCTAJU03PHASRDXRCVMQPCSITCFFZUYORPEKMNKISMFEKW


In [77]:
address = '2 15th St, Ocean City, MD 21842' #Address of Courtyard by Marriot Ocean Front in Ocean City, MD

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

38.344899 -75.07821


In [78]:
search_query = 'Bar'
radius = 500
print(search_query + ' .... OK!')

Bar .... OK!


In [79]:
# URL to find bars within a 5 mile radius from the hotel
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=TTKHJVXTE44UI1ZDXT4VQN52NNAIG1MRF1SDHTUIKSQB1BK0&client_secret=ZGXCTAJU03PHASRDXRCVMQPCSITCFFZUYORPEKMNKISMFEKW&ll=38.344899,-75.07821&v=20180604&query=Bar&radius=500&limit=30'

In [80]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb9cc2ab57e88001c9d897f'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/pool_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d15e941735',
      'name': 'Pool',
      'pluralName': 'Pools',
      'primary': True,
      'shortName': 'Pool'}],
    'hasPerk': False,
    'id': '500713d1e4b080f3fd6a6f5a',
    'location': {'address': '15th St',
     'cc': 'US',
     'city': 'Ocean City',
     'country': 'United States',
     'crossStreet': '1409 Wilmington Ave',
     'distance': 35,
     'formattedAddress': ['15th St (1409 Wilmington Ave)',
      'Ocean City, MD 21842',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.34485434235498,
       'lng': -75.07861006859237}],
     'lat': 38.34485434235498,
     'lng': -75.07861006859237,
     'postalCode': '21842',
     'state': 'MD'},
    'name': '15th St Pool Bar',
    'referralId': 'v-1589234643

In [81]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d15e941735', 'name': 'P...",False,500713d1e4b080f3fd6a6f5a,15th St,US,Ocean City,United States,1409 Wilmington Ave,35,"[15th St (1409 Wilmington Ave), Ocean City, MD...","[{'label': 'display', 'lat': 38.34485434235498...",38.344854,-75.078610,21842,MD,15th St Pool Bar,v-1589234643
1,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,51648371e4b061c02471545b,17 Atlantic Ave,US,Ocean City,United States,NaN,246,"[17 Atlantic Ave, Ocean City, MD 21842, United...","[{'label': 'display', 'lat': 38.34708214785496...",38.347082,-75.077709,21842,MD,Coral Reel Bar & Grill,v-1589234643
2,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,5ce18fcabd4009002c7a6173,1511 Atlantic Ave,US,Ocean City,United States,16th St On The Boards,71,"[1511 Atlantic Ave (16th St On The Boards), Oc...","[{'label': 'display', 'lat': 38.345314, 'lng':...",38.345314,-75.077580,21842,MD,Franco's Pizza & Bar,v-1589234643
3,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,5382a491498eaf0c790aa44c,1804 Philadelphia Ave,US,Ocean City,United States,NaN,192,"[1804 Philadelphia Ave, Ocean City, MD 21842, ...","[{'label': 'display', 'lat': 38.34662067488084...",38.346621,-75.078450,21842,MD,OC's Sushi Bar,v-1589234643
4,"[{'id': '4bf58dd8d48988d1d5941735', 'name': 'H...",False,501358e3e4b0be9af4f46811,NaN,US,Ocean City,United States,NaN,217,"[Ocean City, MD 21842, United States]","[{'label': 'display', 'lat': 38.343097, 'lng':...",38.343097,-75.079183,21842,MD,Safari Bar,v-1589234643


In [82]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,15th St Pool Bar,Pool,15th St,US,Ocean City,United States,1409 Wilmington Ave,35,"[15th St (1409 Wilmington Ave), Ocean City, MD...","[{'label': 'display', 'lat': 38.34485434235498...",38.344854,-75.078610,21842,MD,500713d1e4b080f3fd6a6f5a
1,Coral Reel Bar & Grill,Café,17 Atlantic Ave,US,Ocean City,United States,NaN,246,"[17 Atlantic Ave, Ocean City, MD 21842, United...","[{'label': 'display', 'lat': 38.34708214785496...",38.347082,-75.077709,21842,MD,51648371e4b061c02471545b
2,Franco's Pizza & Bar,Pizza Place,1511 Atlantic Ave,US,Ocean City,United States,16th St On The Boards,71,"[1511 Atlantic Ave (16th St On The Boards), Oc...","[{'label': 'display', 'lat': 38.345314, 'lng':...",38.345314,-75.077580,21842,MD,5ce18fcabd4009002c7a6173
3,OC's Sushi Bar,Sushi Restaurant,1804 Philadelphia Ave,US,Ocean City,United States,NaN,192,"[1804 Philadelphia Ave, Ocean City, MD 21842, ...","[{'label': 'display', 'lat': 38.34662067488084...",38.346621,-75.078450,21842,MD,5382a491498eaf0c790aa44c
4,Safari Bar,Hotel Bar,NaN,US,Ocean City,United States,NaN,217,"[Ocean City, MD 21842, United States]","[{'label': 'display', 'lat': 38.343097, 'lng':...",38.343097,-75.079183,21842,MD,501358e3e4b0be9af4f46811
5,Pool Bar,Hotel Bar,NaN,US,Ocean City,United States,NaN,264,"[Ocean City, MD 21842, United States]","[{'label': 'display', 'lat': 38.34722055, 'lng...",38.347221,-75.078877,21842,MD,4e208ed218a88345f041f0ae
6,Ocean View Grill,American Restaurant,1601 Atlantic Ave,US,Ocean City,United States,16th St. And The Board Walk,114,[1601 Atlantic Ave (16th St. And The Board Wal...,"[{'label': 'display', 'lat': 38.34571127188901...",38.345711,-75.077406,21842,MD,4c632a35ec94a59316262bca
7,Ocean 13,Bar,1301 Atlantic Ave,US,Ocean City,United States,NaN,184,"[1301 Atlantic Ave, Ocean City, MD 21842, Unit...","[{'label': 'display', 'lat': 38.34327912520673...",38.343279,-75.078675,21842,MD,4c001583ca1920a105d0ed81
8,Holiday Inn Hotel & Suites Ocean City,Hotel,1701 Atlantic Ave,US,Ocean City,United States,at Baltimore Ave.,242,"[1701 Atlantic Ave (at Baltimore Ave.), Ocean ...","[{'label': 'display', 'lat': 38.3469862, 'lng'...",38.346986,-75.077400,21842,MD,4c74fad666be6dcb0505bf0f
9,"Amy Barshinger, CRNP",Doctor's Office,1001 Philadelphia Ave,US,Ocean City,United States,NaN,526,"[1001 Philadelphia Ave, Ocean City, MD 21842, ...","[{'label': 'display', 'lat': 38.34163515554201...",38.341635,-75.082578,21842,MD,5a3a9612b9ac38382fcef099


In [83]:
dataframe_filtered.name

0                         15th St Pool Bar
1                   Coral Reel Bar & Grill
2                     Franco's Pizza & Bar
3                           OC's Sushi Bar
4                               Safari Bar
5                                 Pool Bar
6                         Ocean View Grill
7                                 Ocean 13
8    Holiday Inn Hotel & Suites Ocean City
9                     Amy Barshinger, CRNP
Name: name, dtype: object

### Let's Visualize a Map

In [104]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Hotel
folium.features.Marker(
    [latitude, longitude],
    icon = folium.Icon(color="red",icon="fa-hotel", prefix='fa'),
    radius=10,
    color='red',
    popup='Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add potential bar locations as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.Marker(
        [lat, lng],
        icon = folium.Icon(color="blue",icon="fa-beer", prefix='fa'),
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

# Results

In [117]:
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,15th St Pool Bar,Pool,15th St,US,Ocean City,United States,1409 Wilmington Ave,35,"[15th St (1409 Wilmington Ave), Ocean City, MD...","[{'label': 'display', 'lat': 38.34485434235498...",38.344854,-75.078610,21842,MD,500713d1e4b080f3fd6a6f5a
1,Coral Reel Bar & Grill,Café,17 Atlantic Ave,US,Ocean City,United States,NaN,246,"[17 Atlantic Ave, Ocean City, MD 21842, United...","[{'label': 'display', 'lat': 38.34708214785496...",38.347082,-75.077709,21842,MD,51648371e4b061c02471545b
2,Franco's Pizza & Bar,Pizza Place,1511 Atlantic Ave,US,Ocean City,United States,16th St On The Boards,71,"[1511 Atlantic Ave (16th St On The Boards), Oc...","[{'label': 'display', 'lat': 38.345314, 'lng':...",38.345314,-75.077580,21842,MD,5ce18fcabd4009002c7a6173
3,OC's Sushi Bar,Sushi Restaurant,1804 Philadelphia Ave,US,Ocean City,United States,NaN,192,"[1804 Philadelphia Ave, Ocean City, MD 21842, ...","[{'label': 'display', 'lat': 38.34662067488084...",38.346621,-75.078450,21842,MD,5382a491498eaf0c790aa44c
4,Safari Bar,Hotel Bar,NaN,US,Ocean City,United States,NaN,217,"[Ocean City, MD 21842, United States]","[{'label': 'display', 'lat': 38.343097, 'lng':...",38.343097,-75.079183,21842,MD,501358e3e4b0be9af4f46811
5,Pool Bar,Hotel Bar,NaN,US,Ocean City,United States,NaN,264,"[Ocean City, MD 21842, United States]","[{'label': 'display', 'lat': 38.34722055, 'lng...",38.347221,-75.078877,21842,MD,4e208ed218a88345f041f0ae
6,Ocean View Grill,American Restaurant,1601 Atlantic Ave,US,Ocean City,United States,16th St. And The Board Walk,114,[1601 Atlantic Ave (16th St. And The Board Wal...,"[{'label': 'display', 'lat': 38.34571127188901...",38.345711,-75.077406,21842,MD,4c632a35ec94a59316262bca
7,Ocean 13,Bar,1301 Atlantic Ave,US,Ocean City,United States,NaN,184,"[1301 Atlantic Ave, Ocean City, MD 21842, Unit...","[{'label': 'display', 'lat': 38.34327912520673...",38.343279,-75.078675,21842,MD,4c001583ca1920a105d0ed81
8,Holiday Inn Hotel & Suites Ocean City,Hotel,1701 Atlantic Ave,US,Ocean City,United States,at Baltimore Ave.,242,"[1701 Atlantic Ave (at Baltimore Ave.), Ocean ...","[{'label': 'display', 'lat': 38.3469862, 'lng'...",38.346986,-75.077400,21842,MD,4c74fad666be6dcb0505bf0f
9,"Amy Barshinger, CRNP",Doctor's Office,1001 Philadelphia Ave,US,Ocean City,United States,NaN,526,"[1001 Philadelphia Ave, Ocean City, MD 21842, ...","[{'label': 'display', 'lat': 38.34163515554201...",38.341635,-75.082578,21842,MD,5a3a9612b9ac38382fcef099


# Discussion
Looking online, we see that Ocean 13 is the most popular bar among 22-28 year olds.